# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [31]:
import json
import urllib.request
import json
import os
import ssl
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import AmlCompute ,ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azure.ai.ml.entities import Model, Environment


## Dataset

### Overview
I use the IRIS Pieces dataset for this project. It includes three iris species with 50 samples each as well as 4 properties and 1 label (the Species) about each flower.

Properties:
+ 'SepalLengthCm'
+ 'SepalWidthCm'
+ 'PetalLengthCm'
+ 'PetalWidthCm'

Label:'Species'


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'final-experiment'

experiment=Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='IRIS-Dataset')
dataset = dataset.drop_columns("Id")


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

amlcompute_cluster_name = "notebook247476"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',                                                           
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=compute_target,training_data=dataset,
    label_column_name="Species",   
    path = "./",
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=False)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
fitted_model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### Prepare scoring function and MLClient

In [70]:
#Get all code to deploy to the `DEPLOY_DIR`
DEPLOY_DIR = "./deploy/outputs"
# os.mkdir(DEPLOY_DIR)
# best_run.download_files(output_directory=DEPLOY_DIR)

In [ ]:
#Create MLClient object, this object is like a Workspace handler
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ML_CLIENT = MLClient.from_config(DefaultAzureCredential())

In [71]:
# Register the model
from azure.ai.ml.constants import AssetTypes
model_path = os.path.join(DEPLOY_DIR, "model.pkl")
model = Model(
    name="thaihocmodel",
    path=model_path,
    type=AssetTypes.CUSTOM_MODEL,
    description="The best model from AutoML"
)
model = ML_CLIENT.models.create_or_update(model)

### Config an endpoint

In [60]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime
endpoint_name = f"automl-{datetime.datetime.now().strftime('%m%d%H%M%f')}"
ENDPOINT = ManagedOnlineEndpoint(
    name=endpoint_name
)

### Attach the endpoint and model to workspace
    * Register the endpoint
    * Register the model

In [61]:
# Register the end point
ML_CLIENT.begin_create_or_update(ENDPOINT)

In [62]:
import time
time.sleep(5)
while ML_CLIENT.online_endpoints.get(endpoint_name).provisioning_state == "Creating":
    time.sleep(1)

if (ML_CLIENT.online_endpoints.get(endpoint_name).provisioning_state == "Succeeded"):
    print(f"Deploy endpoint {endpoint_name} success")
else:
    print(f"Deploy endpoint {endpoint_name} failed")

Deploy endpoint automl-12311240168597 success


### Create the Deployment object

In [72]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration
deployment_name = "auto-deploy-15"

environment = Environment(image="mcr.microsoft.com/azureml/curated/responsibleai-ubuntu20.04-py38-cpu:38")
env = Environment(
    conda_file=os.path.join(DEPLOY_DIR, "conda_env_v_1_0_0.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
DEPLOYMENT = ManagedOnlineDeployment(
    name=deployment_name,
    environment = env,
    endpoint_name=endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code=DEPLOY_DIR, 
        scoring_script="scoring_file_v_2_0_0.py"
    ),
    instance_count=1,
    instance_type="Standard_DS3_v2"
)

ML_CLIENT.begin_create_or_update(DEPLOYMENT)

Check: endpoint automl-12311240168597 exists
Uploading outputs (0.26 MBs): 100%|██████████| 255688/255688 [00:00<00:00, 2841748.35it/s]




.

In [73]:
import time
while ML_CLIENT.online_deployments.get(name=deployment_name, endpoint_name=endpoint_name).provisioning_state == "Updating":
    time.sleep(1)

if (ML_CLIENT.online_deployments.get(name=deployment_name, endpoint_name=endpoint_name).provisioning_state == "Success"):
    print(f"Deploy endpoint {endpoint_name} success")
else:
    print(f"Deploy endpoint {endpoint_name} failed")

..........................................Deploy endpoint automl-12311240168597 failed
.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:



def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "Inputs": {
    "data": [
      {
        "SepalLengthCm": 5.1,
        "SepalWidthCm": 3.5,
        "PetalLengthCm": 1.4,
        "PetalWidthCm": .2
      }
    ]
  }
}

body = str.encode(json.dumps(data))

url = endpoint.scoring_uri
api_key = ML_CLIENT.online_endpoints.get_keys(endpoint_name)

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key.primary_key), 'azureml-model-deployment': 'automldeployment' }
req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Get log
ML_CLIENT.online_deployments.get_logs(name="AutoMlDeployment", endpoint_name=endpoint_name, lines=1000)

In [ ]:
#delete the service
ML_CLIENT.online_endpoints.begin_delete(name=endpoint_name)

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
